# 航空公司客户价值分析

使用航空公司客户数据，对客户进行分群，比较不同类别客户的价值，从而制定相应的营销策略。

## 一、背景

面对激烈的市场竞争，各个航空公司相继推出了更优惠的营销方式来吸引更多的客户，国内某航空公司面临着常旅客流失、竞争力下降和航空资源未充分利用等经营危机。通过建立合理的客户价值评估模型，对客户进行分类，分析比较不同客户群体的价值，并制定相应的营销策略，对不同的客户群提供个性化的客户服务是必须的和有效的。

总体流程:

1. 抽取航空公司2012年4月1日至2014年3月31日的数据。
2. 对抽取的数据进行数据探索分析与预处理，包括数据缺失值与异常值的探索分析、数据清洗、特征构建、标准化等操作。
3. 针对模型结果得到不同价值的客户，采用不同的营销手段，提供定制化的服务。

## 二、查看数据

抽取观测窗口2012年4月1日至2014年3月31日内有乘机记录的所有客户的详细数据形成历史数据，总共62988条记录,44个属性

## 三、数据清洗

### 导入库

In [1]:
import matplotlib as plt
import numpy as np 
import pandas as pd

### 导入数据

path=r'F:\Data\air_data\air_data.csv'
df=pd.read_csv(path)

### 查看数据

In [4]:
df.head()

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,...,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
0,54993,2006/11/2,2008/12/24,男,6,.,北京,CN,31.0,2014/3/31,...,39992,114452,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50
1,28065,2007/2/19,2007/8/3,男,6,NaN,北京,CN,42.0,2014/3/31,...,12000,53288,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33
2,55106,2007/2/1,2007/8/30,男,6,.,北京,CN,40.0,2014/3/31,...,15491,55202,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26
3,21189,2008/8/22,2008/8/23,男,5,Los Angeles,CA,US,64.0,2014/3/31,...,0,34890,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12
4,39546,2009/4/10,2009/4/15,男,6,贵阳,贵州,CN,48.0,2014/3/31,...,22704,64969,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62988 entries, 0 to 62987
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MEMBER_NO                62988 non-null  int64  
 1   FFP_DATE                 62988 non-null  object 
 2   FIRST_FLIGHT_DATE        62988 non-null  object 
 3   GENDER                   62985 non-null  object 
 4   FFP_TIER                 62988 non-null  int64  
 5   WORK_CITY                60719 non-null  object 
 6   WORK_PROVINCE            59740 non-null  object 
 7   WORK_COUNTRY             62962 non-null  object 
 8   AGE                      62568 non-null  float64
 9   LOAD_TIME                62988 non-null  object 
 10  FLIGHT_COUNT             62988 non-null  int64  
 11  BP_SUM                   62988 non-null  int64  
 12  EP_SUM_YR_1              62988 non-null  int64  
 13  EP_SUM_YR_2              62988 non-null  int64  
 14  SUM_YR_1              

In [6]:
df.describe()

,MEMBER_NO,FFP_TIER,AGE,FLIGHT_COUNT,BP_SUM,EP_SUM_YR_1,EP_SUM_YR_2,SUM_YR_1,SUM_YR_2,SEG_KM_SUM,...,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
count,62988.000000,62988.000000,62568.000000,62988.000000,62988.000000,62988.0,62988.000000,62437.000000,62850.000000,62988.000000,...,62988.000000,62988.000000,62988.000000,62988.0000,62988.000000,62988.000000,62988.000000,62988.000000,62988.000000,62988.000000
mean,31494.500000,4.102162,42.476346,11.839414,10925.081254,0.0,265.689623,5355.376064,5604.026014,17123.878691,...,1355.006223,1620.695847,1080.378882,12545.7771,6638.739585,0.486419,0.513581,0.522293,0.468422,2.728155
std,18183.213715,0.373856,9.885915,14.049471,16339.486151,0.0,1645.702854,8109.450147,8703.364247,20960.844623,...,7868.477000,8294.398955,5639.857254,20507.8167,12601.819863,0.319105,0.319105,0.339632,0.338956,7.364164
min,1.000000,4.000000,6.000000,2.000000,0.000000,0.0,0.000000,0.000000,0.000000,368.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15747.750000,4.000000,35.000000,3.000000,2518.000000,0.0,0.000000,1003.000000,780.000000,4747.000000,...,0.000000,0.000000,0.000000,2775.0000,700.000000,0.250000,0.288889,0.258150,0.167954,0.000000
50%,31494.500000,4.000000,41.000000,7.000000,5700.000000,0.0,0.000000,2800.000000,2773.000000,9994.000000,...,0.000000,0.000000,0.000000,6328.5000,2860.500000,0.500000,0.500000,0.514252,0.476747,0.000000
75%,47241.250000,4.000000,48.000000,15.000000,12831.000000,0.0,0.000000,6574.000000,6845.750000,21271.250000,...,0.000000,345.000000,0.000000,14302.5000,7500.000000,0.711111,0.750000,0.815091,0.728375,1.000000
max,62988.000000,6.000000,110.000000,213.000000,505308.000000,0.0,74460.000000,239560.000000,234188.000000,580717.000000,...,984938.000000,984938.000000,728282.000000,985572.0000,728282.000000,1.000000,1.000000,0.999989,0.999993,140.000000


### 查看空值

In [3]:
df.isnull().sum()

MEMBER_NO                     0
FFP_DATE                      0
FIRST_FLIGHT_DATE             0
GENDER                        3
FFP_TIER                      0
WORK_CITY                  2269
WORK_PROVINCE              3248
WORK_COUNTRY                 26
AGE                         420
LOAD_TIME                     0
FLIGHT_COUNT                  0
BP_SUM                        0
EP_SUM_YR_1                   0
EP_SUM_YR_2                   0
SUM_YR_1                    551
SUM_YR_2                    138
SEG_KM_SUM                    0
WEIGHTED_SEG_KM               0
LAST_FLIGHT_DATE              0
AVG_FLIGHT_COUNT              0
AVG_BP_SUM                    0
BEGIN_TO_FIRST                0
LAST_TO_END                   0
AVG_INTERVAL                  0
MAX_INTERVAL                  0
ADD_POINTS_SUM_YR_1           0
ADD_POINTS_SUM_YR_2           0
EXCHANGE_COUNT                0
avg_discount                  0
P1Y_Flight_Count              0
L1Y_Flight_Count              0
P1Y_BP_S

观察数据发现，原始数据中存在票价为空值、票价最小值为0、折扣率最小值为0、总飞行公里数大于0的记录。票价为空值的数据可能是客户不存在乘机记录造成的。其他的数据可能是客户乘坐0折机票或者积分兑换造成的。由于原始数据量大，这类数据所占比例较小，对于问题影响不大，因此对其进行丢弃处理。同时，在进行数据探索时，发现部分年龄大于100的记录，也进行丢弃处理，具体处理方法如下:

1）丢弃票价为空的记录。 2）保留票价不为0的，或者平均折扣率不为0且总飞行公里数大于0的记录。 3）丢弃年龄大于100的记录

1. 丢弃票价为空的记录

In [7]:
df=df[(df['SUM_YR_1'].notnull())&(df['SUM_YR_2'].notnull())]

2. 保留票价不为0的，或者平均折扣率不为0且总飞行公里数大于0的记录

In [8]:
df=df[((df.SUM_YR_1!=0)|(df.SUM_YR_2!=0))&((df.avg_discount!=0)&(df.SEG_KM_SUM>0))]

3. 去除年龄大于100的记录

In [9]:
df=df[(df.AGE<=100)|(df.AGE.isnull())]

In [11]:
df.shape

(62043, 44)

## 数据可视化

#### 客户基本信息分布分析

选取客户基本信息中的入会时间、性别、会员卡级别和年龄字段进行探索分析，探索客户的基本信息分布情况

1. 各年份会员入会人数

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5iWJpQ2cyEbMpSduzo52V1eRQ6dLYbkNoArH9398cRK.3nsnbPWjuBHtgA.*PIb.aq7.FMHKkGg3B5GL7nCNDRw!/b&bo=zAP7AcwD.wEDCSw!&rf=viewer_4)

2. 会员性别比例

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5jKjbPqA7QemKbYuKQcfzg.w4.DllK*LQHrmJAoyrhKS9YNjXhotNaJ*gDSgXFnxPtlEVKgSy*rOLyl7ZM3VIcE!/b&bo=WAT7AQAAAAADB4Q!&rf=viewer_4)

3. 会员各级别人数

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5qf6MZJEOLug7BJyCMKD2kIVjSr3Dz9Ai8FPdR773Gab1swvNhU*e4GHLzbFi.zKnZem6Kk.D7pEMzjwjY1IfVw!/b&bo=8AD7AfAA.wEDCSw!&rf=viewer_4)

4. 会员年龄分布箱线图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5qf6MZJEOLug7BJyCMKD2kLfTMeuMsCyp6a.Yfd2x*BY3KHQJWh*l71y7ciE1uO.U3J3hXGNNXbpVQ6vmr*ZMoQ!/b&bo=dwBAAncAQAIDCSw!&rf=viewer_4)

#### 小结

可以看出大部分会员年龄集中在30～50岁之间，极少量的会员年龄小于20岁或高于60岁

#### 客户乘机信息分布分析

5. 会员最后乘机至结束时长分布箱线图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5gttgJLvOyeGE66lwaVVDfjs.IcjEX775XtnQVFuk2j7YR84zz*LYqF9or7Qfh1E3DDd7zEpDaUOYFFnW.g0mhg!/b&bo=fgBPAn4ATwIDCSw!&rf=viewer_4)

#### 小结

由图可知: 客户的入会时长主要分布在50～300区间内，另外有一部分客户群体的入会时长分布在600以上的区间，可分为两个群体

6. 会员飞行次数分布箱线图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5qf6MZJEOLug7BJyCMKD2kIMRI0sd2iHF1R7..yqs49YeGHWdU8LhF47sVoA4E*ZUsut1yJkntzA*hnk0eILYm4!/b&bo=dwAzAncAMwIDCSw!&rf=viewer_4)

7. 客户总飞行公里箱线图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5gttgJLvOyeGE66lwaVVDfgf8kqpFQhJ88lftkyKQj3UmeDcjl1YWgyhcSWNpH0PNMnLa2MGucdwClOWz3BX700!/b&bo=fgAXAn4AFwIDCSw!&rf=viewer_4)

#### 小结

客户的飞行次数与总飞行公里数也明显地分为两个群体，大部分客户集中在箱型图下方的箱体中，少数客户分散分布在箱体上界的上方，这部分客户很可能是高价值客户，因为其飞行次数和总飞行公里数明显超过箱体内的其他客户

#### 客户积分信息分布分析

8. 会员兑换积分次数分布直方图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5l5nvf1REAfDadZUMSZlRl*tbHIgCQSjwFlriKk*S6pvIdPwh90BVa8QnYC4mAyY2LTNPO9zwayJRwThGsdgs8s!/b&bo=zAP7AcwD.wEDCSw!&rf=viewer_4)

#### 小结

由图可知，绝大部分客户的兑换次数在0～10的区间内，进行积分兑换的客户很少

9. 客户总累计积分箱线图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5gttgJLvOyeGE66lwaVVDficWXcJLx86zseBTC*LsOINYyLf0JiJtWh*ORjcs1GiKPFSz0yMqk9ZGqQq25YtnaU!/b&bo=hgAXAoYAFwIDCSw!&rf=viewer_4)

#### 小结

绘制会员总累计积分箱线图,一部分客户集中在箱体中，少部分客户分散分布在箱体上方，这部分客户的积分要明显高于箱体内的客户的积分。